In [1]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import matplotlib.pyplot as plt
%matplotlib inline

[INFO] This non-commercial license of GraphLab Create is assigned to caseyoneill78@hotmail.comand will expire on September 26, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1209 - Server binary: /usr/local/lib/python2.7/dist-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1445800576.log
[INFO] GraphLab Server Version: 1.6.1


In [2]:
train_file = 'http://s3.amazonaws.com/dato-datasets/millionsong/10000.txt'

data = gl.SFrame.read_csv(train_file, header=False, delimiter='\t', verbose=False)
data.rename({'X1': 'user_id', 'X2': 'song_id', 'X3': 'listen_count'}).show()

In [3]:
(train_data, test_data) = data.random_split(0.8, seed=1)

In [4]:
popularity_model = gl.popularity_recommender.create(train_data, 'user_id', 'song_id')

PROGRESS: Recsys training: model = popularity
PROGRESS: Warning: Column 'listen_count' ignored.
PROGRESS:     To use this column as the target, set target = "listen_count" and use a method that allows the use of a target.
PROGRESS: Preparing data set.
PROGRESS:     Data has 1599753 observations with 76085 users and 10000 items.
PROGRESS:     Data prepared in: 3.26787s
PROGRESS: 1599753 observations to process; with 10000 unique items.


In [5]:
item_sin_model = gl.item_similarity_recommender.create(train_data, 'user_id', 'song_id')

PROGRESS: Recsys training: model = item_similarity
PROGRESS: Warning: Column 'listen_count' ignored.
PROGRESS:     To use this column as the target, set target = "listen_count" and use a method that allows the use of a target.
PROGRESS: Preparing data set.
PROGRESS:     Data has 1599753 observations with 76085 users and 10000 items.
PROGRESS:     Data prepared in: 3.54649s
PROGRESS: Computing item similarity statistics:
PROGRESS: Computing most similar items for 10000 items:
PROGRESS: +-----------------+-----------------+
PROGRESS: | Number of items | Elapsed Time    |
PROGRESS: +-----------------+-----------------+
PROGRESS: | 1000            | 2.74853         |
PROGRESS: | 2000            | 2.94093         |
PROGRESS: | 3000            | 3.13237         |
PROGRESS: | 4000            | 3.31997         |
PROGRESS: | 5000            | 3.5033          |
PROGRESS: | 6000            | 3.68188         |
PROGRESS: | 7000            | 3.85678         |
PROGRESS: | 8000            | 4.02821   

In [6]:
result = gl.recommender.util.compare_models(test_data, [popularity_model, item_sin_model],
                                           user_sample=.1, skip_set=train_data)

compare_models: using 6871 users to estimate model performance
PROGRESS: Evaluate model M0
PROGRESS: recommendations finished on 1000/6871 queries. users per second: 1515.63
PROGRESS: recommendations finished on 2000/6871 queries. users per second: 1505.65
PROGRESS: recommendations finished on 3000/6871 queries. users per second: 1494.55
PROGRESS: recommendations finished on 4000/6871 queries. users per second: 1499.49
PROGRESS: recommendations finished on 5000/6871 queries. users per second: 1484.17
PROGRESS: recommendations finished on 6000/6871 queries. users per second: 1472.68

Precision and recall summary statistics by cutoff
+--------+-----------------+------------------+
| cutoff |  mean_precision |   mean_recall    |
+--------+-----------------+------------------+
|   1    |  0.032455246689 | 0.00691333920427 |
|   2    | 0.0342017173628 | 0.0144566776809  |
|   3    | 0.0318245767234 | 0.0197170598761  |
|   4    | 0.0298719254839 | 0.0246706144126  |
|   5    | 0.02788531509

[WARNING] Model trained without a target. Skipping RMSE computation.



PROGRESS: Evaluate model M1
PROGRESS: recommendations finished on 1000/6871 queries. users per second: 733.982
PROGRESS: recommendations finished on 2000/6871 queries. users per second: 737.743
PROGRESS: recommendations finished on 3000/6871 queries. users per second: 740.46
PROGRESS: recommendations finished on 4000/6871 queries. users per second: 742.459
PROGRESS: recommendations finished on 5000/6871 queries. users per second: 741.585
PROGRESS: recommendations finished on 6000/6871 queries. users per second: 740.538

Precision and recall summary statistics by cutoff

[WARNING] Model trained without a target. Skipping RMSE computation.



+--------+-----------------+-----------------+
| cutoff |  mean_precision |   mean_recall   |
+--------+-----------------+-----------------+
|   1    |  0.223548246252 | 0.0595257875546 |
|   2    |  0.189710376947 | 0.0936763451355 |
|   3    |  0.166108766313 |  0.116813204843 |
|   4    |  0.147467617523 |  0.13373462604  |
|   5    |  0.134420026197 |  0.150024302393 |
|   6    |  0.123902391695 |  0.162555697908 |
|   7    |  0.115932386635 |  0.175551174799 |
|   8    |  0.109063455101 |  0.185324227821 |
|   9    |  0.103138795905 |  0.19593731584  |
|   10   | 0.0979187891137 |  0.204741157665 |
+--------+-----------------+-----------------+
[10 rows x 3 columns]



In [7]:
K = 10
users = gl.SArray(data['user_id'].unique().head(100))

In [8]:
rec = item_sin_model.recommend(users=users, k=K)
rec.head()

user_id,song_id,score,rank
b048033af070b5dbb18d5d0e5f334c9390611b04 ...,SOKEEKD12AB0188B39,0.0323530090129,1
b048033af070b5dbb18d5d0e5f334c9390611b04 ...,SOXWYZP12AF72A42A6,0.0251230144724,2
b048033af070b5dbb18d5d0e5f334c9390611b04 ...,SODXRTY12AB0180F3B,0.0247880750827,3
b048033af070b5dbb18d5d0e5f334c9390611b04 ...,SOHTKMO12AB01843B0,0.0238842810871,4
b048033af070b5dbb18d5d0e5f334c9390611b04 ...,SOYYJFK12AB0188DAB,0.0238824101069,5
b048033af070b5dbb18d5d0e5f334c9390611b04 ...,SOASXQD12AB018902F,0.0224881583452,6
b048033af070b5dbb18d5d0e5f334c9390611b04 ...,SOAVTFX12AB0188D87,0.0222954734219,7
b048033af070b5dbb18d5d0e5f334c9390611b04 ...,SOOWVNN12A8C140775,0.0210031363563,8
b048033af070b5dbb18d5d0e5f334c9390611b04 ...,SOFRQTD12A81C233C0,0.0206322056899,9
b048033af070b5dbb18d5d0e5f334c9390611b04 ...,SORDHDA12AB0186828,0.0202977908794,10


In [10]:
# It's time to merge the metadata
songs = gl.SFrame.read_csv('http://s3.amazonaws.com/dato-datasets/millionsong/song_data.csv', verbose=False)
songs = songs[['song_id', 'title', 'artist_name']]
results = rec.join(songs, on='song_id', how='inner')

userset = frozenset(users)
ix = data['user_id'].apply(lambda x: x in userset, int)
user_data = data[ix]
user_data = user_data.join(songs, on='song_id')[['user_id', 'title', 'artist_name']]

In [13]:
# Now let's print out some recommendations
for i in range(25):
    user = list(users)[i]
    print "User: " + str(i + 1)
    user_obs = user_data[user_data['user_id'] == user].head(K)
    del user_obs['user_id']
    user_recs = results[results['user_id'] == str(user)][['title', 'artist_name']]
    
    print "We were told that the user liked these songs: "
    print user_obs.head(K)
    
    print "We recommend these other songs: "
    print user_recs.head(K)
    
    print ""

User: 1
We were told that the user liked these songs: 
+--------------------------------+---------------------------+
|             title              |        artist_name        |
+--------------------------------+---------------------------+
|      You've Got The Love       |   Florence + The Machine  |
| Unite (2009 Digital Remaster)  |        Beastie Boys       |
| Behind The Sea [Live In Ch...  |     Panic At The Disco    |
|  Pure Morning (Album Version)  |          Placebo          |
|   Pata Pata (Album Version)    |       Miriam Makeba       |
| Dog Days Are Over (Radio Edit) |   Florence + The Machine  |
|   Jigsaw Falling Into Place    |         Radiohead         |
|           Just Dance           | Lady GaGa / Colby O'Donis |
|          Silver Soul           |        Beach House        |
|            Monster             |         Lady GaGa         |
+--------------------------------+---------------------------+
[10 rows x 2 columns]

We recommend these other songs: 
+------